# PDX Drug Sensitivity Prediction - Demonstration

## Based on: μPharma - A Microfluidic AI-driven Pharmacotyping Platform

This demo notebook demonstrates our PDX-specific drug sensitivity prediction methodology using synthetic data that mimics the structure of real T-ALL PDX models.

**Key Features:**
- Drug-specific feature engineering (LCK for Dasatinib, BCL2 for Venetoclax)
- PDX-aware rotational cross-validation
- XGBoost with drug-specific hyperparameters
- SHAP-based interpretability
- UMAP visualization from SHAP values

**Authors:** Huiqian Hu, Alphonsus H. C. Ng, Yue Lu

## 1. Setup and Import Libraries

In [ ]:
# Install required packages if needed
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of required packages
required_packages = [
    'numpy', 'pandas', 'scikit-learn', 'xgboost', 
    'matplotlib', 'seaborn', 'shap', 'umap-learn', 'imbalanced-learn'
]

# Check and install missing packages
for package in required_packages:
    try:
        __import__(package.replace('-', '_'))
    except ImportError:
        print(f"Installing {package}...")
        install(package)

# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import (
    roc_auc_score, roc_curve, confusion_matrix, 
    accuracy_score, f1_score, precision_score, recall_score
)
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
import shap
import umap

# Settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
np.random.seed(42)

print("✅ All libraries imported successfully!")
print(f"XGBoost version: {xgboost.__version__}")
print(f"SHAP version: {shap.__version__}")

## 2. Generate Synthetic PDX Data

We create synthetic data that mimics the structure of real PDX experiments with 14 PDX groups.

In [ ]:
def generate_synthetic_pdx_data(n_samples_per_group=150, random_state=42):
    """
    Generate synthetic PDX data mimicking real experimental structure.
    
    Creates data for 14 PDX groups with morphological and protein expression features.
    """
    np.random.seed(random_state)
    
    # Define 14 PDX groups (matching manuscript)
    pdx_groups = ['SJ65', 'SJ49', 'SJ53', 'SJ52', 'SJ7', 
                  '1054', '145', '741', '748', '758', 
                  '2176', '4404', '4406', '4426']
    
    all_data = []
    
    for group_idx, group_name in enumerate(pdx_groups):
        # Each PDX group has slightly different characteristics
        group_offset = group_idx * 0.1
        
        for _ in range(n_samples_per_group):
            # Morphological features (AreaShape)
            area = np.random.normal(100 + group_offset*10, 15)
            perimeter = np.random.normal(40 + group_offset*5, 5)
            form_factor = np.random.uniform(0.5, 0.9)
            eccentricity = np.random.uniform(0.3, 0.8)
            solidity = np.random.uniform(0.7, 1.0)
            extent = np.random.uniform(0.5, 0.85)
            
            # Protein expression features
            # LCK pathway (relevant for Dasatinib)
            lck_base = np.random.lognormal(3.0 + group_offset, 0.5)
            plck_ratio = np.random.beta(2, 5)  # Phosphorylation ratio
            
            intensity_mean_lck = lck_base
            intensity_mean_plck = lck_base * plck_ratio
            intensity_lower_plck = intensity_mean_plck * np.random.uniform(0.7, 0.9)
            intensity_upper_plck = intensity_mean_plck * np.random.uniform(1.1, 1.3)
            
            # BCL2 pathway (relevant for Venetoclax)
            bcl2_base = np.random.lognormal(2.8 + group_offset*0.8, 0.4)
            pbcl2_ratio = np.random.beta(3, 4)
            
            intensity_mean_bcl2 = bcl2_base
            intensity_mean_pbcl2 = bcl2_base * pbcl2_ratio
            intensity_upper_pbcl2 = intensity_mean_pbcl2 * np.random.uniform(1.05, 1.25)
            intensity_min_edge_bcl2 = intensity_mean_bcl2 * np.random.uniform(0.5, 0.8)
            
            # Drug sensitivity (binary outcome)
            # Dasatinib sensitivity correlates with pLCK/LCK ratio
            dasatinib_score = plck_ratio + eccentricity*0.3 + np.random.normal(0, 0.2)
            dasatinib_sensitive = int(dasatinib_score > 0.5)
            
            # Venetoclax sensitivity correlates with pBCL2/BCL2 ratio
            venetoclax_score = pbcl2_ratio + solidity*0.2 + np.random.normal(0, 0.2)
            venetoclax_sensitive = int(venetoclax_score > 0.55)
            
            sample = {
                'Group': group_name,
                'Sample_Type': 'PDX',
                
                # Morphological features
                'AreaShape_Area': area,
                'AreaShape_Perimeter': perimeter,
                'AreaShape_FormFactor': form_factor,
                'AreaShape_Eccentricity': eccentricity,
                'AreaShape_Solidity': solidity,
                'AreaShape_Extent': extent,
                
                # LCK features
                'Intensity_MeanIntensity_LCK': intensity_mean_lck,
                'Intensity_MeanIntensity_pLCK': intensity_mean_plck,
                'Intensity_LowerQuartileIntensity_pLCK': intensity_lower_plck,
                'Intensity_UpperQuartileIntensity_pLCK': intensity_upper_plck,
                
                # BCL2 features
                'Intensity_MeanIntensity_BCL_2': intensity_mean_bcl2,
                'Intensity_MeanIntensity_pBCL2': intensity_mean_pbcl2,
                'Intensity_UpperQuartileIntensity_pBCL2': intensity_upper_pbcl2,
                'Intensity_MinIntensityEdge_BCL_2': intensity_min_edge_bcl2,
                
                # Drug sensitivities
                'Dasatinib Sensitivity': dasatinib_sensitive,
                'Venetoclax Sensitivity': venetoclax_sensitive
            }
            
            all_data.append(sample)
    
    return pd.DataFrame(all_data)

# Generate data
print("Generating synthetic PDX data...")
pdx_df = generate_synthetic_pdx_data(n_samples_per_group=150)

print(f"\n✅ Generated dataset with {len(pdx_df)} samples")
print(f"Number of PDX groups: {pdx_df['Group'].nunique()}")
print(f"\nPDX groups: {', '.join(pdx_df['Group'].unique())}")
print(f"\nClass distribution:")
print(f"Dasatinib - Sensitive: {pdx_df['Dasatinib Sensitivity'].sum()}, Resistant: {len(pdx_df) - pdx_df['Dasatinib Sensitivity'].sum()}")
print(f"Venetoclax - Sensitive: {pdx_df['Venetoclax Sensitivity'].sum()}, Resistant: {len(pdx_df) - pdx_df['Venetoclax Sensitivity'].sum()}")

# Display first few rows
pdx_df.head()

## 3. Feature Engineering

Create drug-specific engineered features including ratios, log transformations, and interaction terms.

In [ ]:
def engineer_features(df):
    """
    Engineer features following the manuscript methodology.
    """
    df = df.copy()
    
    # Basic ratio features
    df['pLCK_to_LCK_ratio'] = np.divide(
        df['Intensity_MeanIntensity_pLCK'].values,
        df['Intensity_MeanIntensity_LCK'].values,
        out=np.zeros_like(df['Intensity_MeanIntensity_pLCK'].values, dtype=float),
        where=df['Intensity_MeanIntensity_LCK'].values!=0
    )
    
    df['pBCL2_to_BCL2_ratio'] = np.divide(
        df['Intensity_MeanIntensity_pBCL2'].values,
        df['Intensity_MeanIntensity_BCL_2'].values,
        out=np.zeros_like(df['Intensity_MeanIntensity_pBCL2'].values, dtype=float),
        where=df['Intensity_MeanIntensity_BCL_2'].values!=0
    )
    
    # Log transformations
    df['log_pLCK'] = np.log1p(df['Intensity_MeanIntensity_pLCK'])
    df['log_LCK'] = np.log1p(df['Intensity_MeanIntensity_LCK'])
    df['log_pBCL2'] = np.log1p(df['Intensity_MeanIntensity_pBCL2'])
    df['log_BCL2'] = np.log1p(df['Intensity_MeanIntensity_BCL_2'])
    
    # Log ratios
    df['log_pLCK_to_LCK_ratio'] = df['log_pLCK'] - df['log_LCK']
    df['log_pBCL2_to_BCL2_ratio'] = df['log_pBCL2'] - df['log_BCL2']
    
    # Interaction features with morphology
    df['pLCK_x_AreaShape_Area'] = df['Intensity_MeanIntensity_pLCK'] * df['AreaShape_Area']
    df['pBCL2_x_AreaShape_Area'] = df['Intensity_MeanIntensity_pBCL2'] * df['AreaShape_Area']
    
    # Z-scores within PDX groups
    for group in df['Group'].unique():
        group_mask = df['Group'] == group
        if sum(group_mask) > 1:
            group_data = df.loc[group_mask, 'Intensity_MeanIntensity_pLCK']
            df.loc[group_mask, 'pLCK_zscore'] = (group_data - group_data.mean()) / (group_data.std() + 1e-6)
            
            group_data = df.loc[group_mask, 'Intensity_MeanIntensity_pBCL2']
            df.loc[group_mask, 'pBCL2_zscore'] = (group_data - group_data.mean()) / (group_data.std() + 1e-6)
    
    # Fill any missing z-scores
    df['pLCK_zscore'] = df['pLCK_zscore'].fillna(0)
    df['pBCL2_zscore'] = df['pBCL2_zscore'].fillna(0)
    
    return df

# Apply feature engineering
print("Applying feature engineering...")
pdx_df = engineer_features(pdx_df)

# Define drug-specific feature sets
lck_features = [col for col in pdx_df.columns if 'LCK' in col.upper() and 'BCL' not in col.upper()]
bcl2_features = [col for col in pdx_df.columns if ('BCL' in col.upper() or 'pBCL2' in col) and 'LCK' not in col.upper()]
shape_features = [col for col in pdx_df.columns if 'AreaShape' in col]

dasatinib_features = list(set(lck_features + shape_features))
venetoclax_features = list(set(bcl2_features + shape_features))

print(f"\n✅ Feature engineering completed!")
print(f"Dasatinib features: {len(dasatinib_features)}")
print(f"Venetoclax features: {len(venetoclax_features)}")
print(f"\nSample engineered features:")
print(f"  - pLCK/LCK ratio: {pdx_df['pLCK_to_LCK_ratio'].mean():.3f} ± {pdx_df['pLCK_to_LCK_ratio'].std():.3f}")
print(f"  - pBCL2/BCL2 ratio: {pdx_df['pBCL2_to_BCL2_ratio'].mean():.3f} ± {pdx_df['pBCL2_to_BCL2_ratio'].std():.3f}")

## 4. Define PDX Rotations

We use predefined PDX rotations that ensure balanced training/validation/test splits.

In [ ]:
# Define Dasatinib rotations (from manuscript)
dasatinib_rotations = [
    {
        'train': np.array(['SJ65', 'SJ49', 'SJ53', 'SJ52', '1054', '145', '741', '748', '4406']),
        'val': np.array(['SJ7', '4426']),
        'test': np.array(['758', '2176', '4404'])
    },
    {
        'train': np.array(['SJ65', 'SJ49', 'SJ52', '4426', '4404', '4406', '741', '758', '2176']),
        'val': np.array(['145', '748']),
        'test': np.array(['SJ7', 'SJ53', '1054'])
    }
]

# Define Venetoclax rotations (from manuscript)
venetoclax_rotations = [
    {
        'train': np.array(['SJ65', 'SJ7', '741', '4404', '748', 'SJ52', '1054', '4406', 'SJ49']),
        'val': np.array(['4426', 'SJ53']),
        'test': np.array(['2176', '758', '145'])
    },
    {
        'train': np.array(['SJ65', 'SJ7', 'SJ49', '4406', '4404', '741', '758', '2176', '145']),
        'val': np.array(['1054', '4426']),
        'test': np.array(['748', 'SJ52', 'SJ53'])
    }
]

print("PDX Rotation Setup:")
print(f"\nDasatinib - Rotation 1:")
print(f"  Training: {', '.join(dasatinib_rotations[0]['train'])}")
print(f"  Validation: {', '.join(dasatinib_rotations[0]['val'])}")
print(f"  Testing: {', '.join(dasatinib_rotations[0]['test'])}")

print(f"\nVenetoclax - Rotation 1:")
print(f"  Training: {', '.join(venetoclax_rotations[0]['train'])}")
print(f"  Validation: {', '.join(venetoclax_rotations[0]['val'])}")
print(f"  Testing: {', '.join(venetoclax_rotations[0]['test'])}")

## 5. Model Training and Evaluation Functions

In [ ]:
def train_and_evaluate_xgboost(X_train, y_train, X_val, y_val, X_test, y_test, 
                               feature_names, drug_name="", k_best=30, xgb_params=None):
    """
    Train XGBoost model with feature selection and evaluation.
    """
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)
    
    # Handle class imbalance with SMOTE if needed
    if pd.Series(y_train).value_counts().min() / pd.Series(y_train).value_counts().max() < 0.5:
        smote = SMOTE(random_state=42)
        X_train_scaled, y_train = smote.fit_resample(X_train_scaled, y_train)
    
    # Feature selection
    selector = SelectKBest(f_classif, k=min(k_best, X_train.shape[1]))
    X_train_selected = selector.fit_transform(X_train_scaled, y_train)
    X_val_selected = selector.transform(X_val_scaled)
    X_test_selected = selector.transform(X_test_scaled)
    
    # Get selected feature names
    selected_indices = selector.get_support(indices=True)
    selected_features = [feature_names[i] for i in selected_indices]
    
    # Define XGBoost model
    if xgb_params is None:
        xgb_params = {
            'n_estimators': 100,
            'learning_rate': 0.05,
            'max_depth': 5,
            'subsample': 0.8,
            'colsample_bytree': 0.8
        }
    
    model = XGBClassifier(
        **xgb_params,
        random_state=42,
        use_label_encoder=False,
        eval_metric='logloss'
    )
    
    # Train model
    model.fit(
        X_train_selected, y_train,
        eval_set=[(X_val_selected, y_val)],
        verbose=False
    )
    
    # Make predictions
    y_pred = model.predict(X_test_selected)
    y_pred_proba = model.predict_proba(X_test_selected)[:, 1]
    
    # Calculate metrics
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'roc_auc': roc_auc_score(y_test, y_pred_proba),
        'f1_score': f1_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred)
    }
    
    return {
        'model': model,
        'scaler': scaler,
        'selector': selector,
        'selected_features': selected_features,
        'metrics': metrics,
        'y_test': y_test,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba,
        'X_test_selected': X_test_selected
    }

print("✅ Model training function defined")

## 6. Run Cross-Validation for Both Drugs

In [ ]:
# Define drug-specific hyperparameters
dasatinib_params = {
    'n_estimators': 100,
    'learning_rate': 0.05,
    'max_depth': 6,
    'min_child_weight': 3,
    'gamma': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8
}

venetoclax_params = {
    'n_estimators': 100,
    'learning_rate': 0.05,
    'max_depth': 5,
    'min_child_weight': 3,
    'gamma': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8
}

# Prepare data
X_das = pdx_df[dasatinib_features]
y_das = pdx_df['Dasatinib Sensitivity']

X_ven = pdx_df[venetoclax_features]
y_ven = pdx_df['Venetoclax Sensitivity']

# Store results
cv_results = {'dasatinib': [], 'venetoclax': []}

print("="*60)
print("DASATINIB CROSS-VALIDATION")
print("="*60)

for rot_idx, rotation in enumerate(dasatinib_rotations):
    print(f"\nRotation {rot_idx+1}:")
    
    # Get indices
    train_idx = pdx_df[pdx_df['Group'].isin(rotation['train'])].index
    val_idx = pdx_df[pdx_df['Group'].isin(rotation['val'])].index
    test_idx = pdx_df[pdx_df['Group'].isin(rotation['test'])].index
    
    # Train and evaluate
    result = train_and_evaluate_xgboost(
        X_das.loc[train_idx], y_das.loc[train_idx],
        X_das.loc[val_idx], y_das.loc[val_idx],
        X_das.loc[test_idx], y_das.loc[test_idx],
        dasatinib_features,
        drug_name="Dasatinib",
        k_best=20,
        xgb_params=dasatinib_params
    )
    
    cv_results['dasatinib'].append(result)
    
    print(f"  Accuracy: {result['metrics']['accuracy']:.3f}")
    print(f"  ROC AUC: {result['metrics']['roc_auc']:.3f}")
    print(f"  F1 Score: {result['metrics']['f1_score']:.3f}")

print("\n" + "="*60)
print("VENETOCLAX CROSS-VALIDATION")
print("="*60)

for rot_idx, rotation in enumerate(venetoclax_rotations):
    print(f"\nRotation {rot_idx+1}:")
    
    # Get indices
    train_idx = pdx_df[pdx_df['Group'].isin(rotation['train'])].index
    val_idx = pdx_df[pdx_df['Group'].isin(rotation['val'])].index
    test_idx = pdx_df[pdx_df['Group'].isin(rotation['test'])].index
    
    # Train and evaluate
    result = train_and_evaluate_xgboost(
        X_ven.loc[train_idx], y_ven.loc[train_idx],
        X_ven.loc[val_idx], y_ven.loc[val_idx],
        X_ven.loc[test_idx], y_ven.loc[test_idx],
        venetoclax_features,
        drug_name="Venetoclax",
        k_best=20,
        xgb_params=venetoclax_params
    )
    
    cv_results['venetoclax'].append(result)
    
    print(f"  Accuracy: {result['metrics']['accuracy']:.3f}")
    print(f"  ROC AUC: {result['metrics']['roc_auc']:.3f}")
    print(f"  F1 Score: {result['metrics']['f1_score']:.3f}")

## 7. Visualization: Performance Comparison

In [ ]:
# Plot ROC curves for best rotation of each drug
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Best Dasatinib rotation (highest AUC)
best_das_idx = np.argmax([r['metrics']['roc_auc'] for r in cv_results['dasatinib']])
best_das = cv_results['dasatinib'][best_das_idx]

fpr, tpr, _ = roc_curve(best_das['y_test'], best_das['y_pred_proba'])
axes[0].plot(fpr, tpr, color='darkorange', lw=2,
             label=f'ROC curve (AUC = {best_das["metrics"]["roc_auc"]:.3f})')
axes[0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
axes[0].set_xlim([0.0, 1.0])
axes[0].set_ylim([0.0, 1.05])
axes[0].set_xlabel('False Positive Rate')
axes[0].set_ylabel('True Positive Rate')
axes[0].set_title('Dasatinib - Best Rotation ROC Curve')
axes[0].legend(loc="lower right")
axes[0].grid(True, alpha=0.3)

# Best Venetoclax rotation
best_ven_idx = np.argmax([r['metrics']['roc_auc'] for r in cv_results['venetoclax']])
best_ven = cv_results['venetoclax'][best_ven_idx]

fpr, tpr, _ = roc_curve(best_ven['y_test'], best_ven['y_pred_proba'])
axes[1].plot(fpr, tpr, color='darkgreen', lw=2,
             label=f'ROC curve (AUC = {best_ven["metrics"]["roc_auc"]:.3f})')
axes[1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
axes[1].set_xlim([0.0, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('Venetoclax - Best Rotation ROC Curve')
axes[1].legend(loc="lower right")
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Best Dasatinib Rotation: {best_das_idx+1} (AUC: {best_das['metrics']['roc_auc']:.3f})")
print(f"Best Venetoclax Rotation: {best_ven_idx+1} (AUC: {best_ven['metrics']['roc_auc']:.3f})")

## 8. SHAP Analysis for Model Interpretability

In [ ]:
# SHAP analysis for best models
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Dasatinib SHAP
explainer_das = shap.TreeExplainer(best_das['model'])
shap_values_das = explainer_das.shap_values(best_das['X_test_selected'])

# Feature importance bar plot
plt.sca(axes[0, 0])
shap.summary_plot(shap_values_das, best_das['X_test_selected'],
                   feature_names=best_das['selected_features'],
                   plot_type="bar", show=False, max_display=10)
axes[0, 0].set_title('Dasatinib - SHAP Feature Importance')

# Summary plot
plt.sca(axes[0, 1])
shap.summary_plot(shap_values_das, best_das['X_test_selected'],
                   feature_names=best_das['selected_features'],
                   show=False, max_display=10)
axes[0, 1].set_title('Dasatinib - SHAP Summary')

# Venetoclax SHAP
explainer_ven = shap.TreeExplainer(best_ven['model'])
shap_values_ven = explainer_ven.shap_values(best_ven['X_test_selected'])

# Feature importance bar plot
plt.sca(axes[1, 0])
shap.summary_plot(shap_values_ven, best_ven['X_test_selected'],
                   feature_names=best_ven['selected_features'],
                   plot_type="bar", show=False, max_display=10)
axes[1, 0].set_title('Venetoclax - SHAP Feature Importance')

# Summary plot
plt.sca(axes[1, 1])
shap.summary_plot(shap_values_ven, best_ven['X_test_selected'],
                   feature_names=best_ven['selected_features'],
                   show=False, max_display=10)
axes[1, 1].set_title('Venetoclax - SHAP Summary')

plt.tight_layout()
plt.show()

print("\nTop 5 Features by SHAP Importance:")
print("\nDasatinib:")
shap_importance_das = np.abs(shap_values_das).mean(0)
for i in np.argsort(shap_importance_das)[-5:][::-1]:
    print(f"  {best_das['selected_features'][i]}: {shap_importance_das[i]:.3f}")

print("\nVenetoclax:")
shap_importance_ven = np.abs(shap_values_ven).mean(0)
for i in np.argsort(shap_importance_ven)[-5:][::-1]:
    print(f"  {best_ven['selected_features'][i]}: {shap_importance_ven[i]:.3f}")

## 9. UMAP Visualization from SHAP Values

In [ ]:
# Create UMAP embeddings from SHAP values
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Dasatinib UMAP
reducer = umap.UMAP(random_state=42, n_neighbors=15)
embedding_das = reducer.fit_transform(shap_values_das)

# Get test PDX groups for coloring
test_groups_das = pdx_df.loc[pdx_df['Group'].isin(dasatinib_rotations[best_das_idx]['test']), 'Group']
test_idx_das = pdx_df[pdx_df['Group'].isin(dasatinib_rotations[best_das_idx]['test'])].index
groups_for_plot = pdx_df.loc[test_idx_das, 'Group'].values

# Plot colored by sensitivity
scatter = axes[0].scatter(embedding_das[:, 0], embedding_das[:, 1],
                         c=best_das['y_test'], cmap='coolwarm',
                         s=100, alpha=0.7)
axes[0].set_xlabel('UMAP 1')
axes[0].set_ylabel('UMAP 2')
axes[0].set_title('Dasatinib - UMAP from SHAP Values')
plt.colorbar(scatter, ax=axes[0], label='Sensitivity')

# Venetoclax UMAP
embedding_ven = reducer.fit_transform(shap_values_ven)

scatter = axes[1].scatter(embedding_ven[:, 0], embedding_ven[:, 1],
                         c=best_ven['y_test'], cmap='coolwarm',
                         s=100, alpha=0.7)
axes[1].set_xlabel('UMAP 1')
axes[1].set_ylabel('UMAP 2')
axes[1].set_title('Venetoclax - UMAP from SHAP Values')
plt.colorbar(scatter, ax=axes[1], label='Sensitivity')

plt.tight_layout()
plt.show()

print("✅ UMAP visualization from SHAP values completed!")
print("\nThe UMAP embeddings show how samples cluster based on their SHAP feature contributions,")
print("providing insights into the decision boundaries learned by the models.")

## 10. Summary and Conclusions

In [ ]:
# Calculate average metrics across rotations
das_metrics = pd.DataFrame([r['metrics'] for r in cv_results['dasatinib']])
ven_metrics = pd.DataFrame([r['metrics'] for r in cv_results['venetoclax']])

print("="*60)
print("CROSS-VALIDATION SUMMARY")
print("="*60)

print("\nDasatinib Performance (Mean ± Std):")
for metric in das_metrics.columns:
    print(f"  {metric}: {das_metrics[metric].mean():.3f} ± {das_metrics[metric].std():.3f}")

print("\nVenetoclax Performance (Mean ± Std):")
for metric in ven_metrics.columns:
    print(f"  {metric}: {ven_metrics[metric].mean():.3f} ± {ven_metrics[metric].std():.3f}")

# Feature consistency analysis
das_features_all = [r['selected_features'] for r in cv_results['dasatinib']]
ven_features_all = [r['selected_features'] for r in cv_results['venetoclax']]

# Count feature frequency
from collections import Counter
das_feature_counts = Counter([f for features in das_features_all for f in features])
ven_feature_counts = Counter([f for features in ven_features_all for f in features])

print("\n" + "="*60)
print("FEATURE CONSISTENCY ACROSS ROTATIONS")
print("="*60)

print("\nMost Consistent Dasatinib Features:")
for feature, count in das_feature_counts.most_common(5):
    print(f"  {feature}: selected in {count}/{len(dasatinib_rotations)} rotations")

print("\nMost Consistent Venetoclax Features:")
for feature, count in ven_feature_counts.most_common(5):
    print(f"  {feature}: selected in {count}/{len(venetoclax_rotations)} rotations")

print("\n" + "="*60)
print("KEY FINDINGS")
print("="*60)
print("""
1. Drug-Specific Feature Selection:
   - Dasatinib models rely primarily on LCK-related features
   - Venetoclax models rely primarily on BCL2-related features
   - Both drugs utilize morphological features (AreaShape)

2. PDX Rotation Importance:
   - Model performance varies with different PDX group assignments
   - Optimal rotations are drug-specific
   - Cross-validation ensures robust performance estimates

3. Model Interpretability:
   - SHAP analysis reveals feature importance and interactions
   - UMAP visualization shows clear separation of sensitive/resistant samples
   - Top features align with known drug mechanisms of action

This demonstration validates our PDX-specific approach to drug sensitivity prediction.
""")